# Importtation de nos librairie

In [15]:
import requests
import json
import pandas as pd
import mysql.connector
import os

# Collecte des donnees

In [8]:
API_KEY = "9f868310439f0a0554273d8f8768c83e"
CITIES = ["Dakar", "Thies", "Diourbel", "Fatick", "Saint Louis", "Louga", "Matam", "Kolda", "Kedougou", "Sedhiou", "Kaffrine", "Ziguinchor", "Kaolack", "Tambacounda"]  # Liste de villes


def fetch_weather_data(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        weather_data = response.json()
        with open(f"data/{city}.json", "w") as json_file:
            json.dump(weather_data, json_file, indent=4)
        print(f"Weather data for {city} saved.")
    else:
        print(f"Error fetching data for {city}")


# Parcourir la liste de villes et appeler fetch_weather_data pour chaque ville
for city in CITIES:
    fetch_weather_data(city)

Weather data for Dakar saved.
Weather data for Thies saved.
Weather data for Diourbel saved.
Weather data for Fatick saved.
Weather data for Saint Louis saved.
Weather data for Louga saved.
Weather data for Matam saved.
Weather data for Kolda saved.
Weather data for Kedougou saved.
Weather data for Sedhiou saved.
Weather data for Kaffrine saved.
Weather data for Ziguinchor saved.
Weather data for Kaolack saved.
Weather data for Tambacounda saved.


# Transformation des donnees

In [9]:
def clean_weather_data(file_path):
    # Charger les données JSON manuellement
    with open(file_path, "r") as file:
        data = json.load(file)

    # Extraire les colonnes d'intérêt avec gestion des sous-dictionnaires
    cleaned_data = {
        "city": data.get("name", None),
        "temperature": data.get("main", {}).get("temp", None),
        "humidity": data.get("main", {}).get("humidity", None),
        "pressure": data.get("main", {}).get("pressure", None),
        "weather_condition": data.get("weather", [{}])[0].get(
            "description", None
        ),  # Premier élément de la liste 'weather'
        "wind_speed": data.get("wind", {}).get("speed", None),
        "wind_deg": data.get("wind", {}).get("deg", None),
        "clouds": data.get("clouds", {}).get(
            "all", None
        ),  # Pourcentage de couverture nuageuse
        "latitude": data.get("coord", {}).get("lat", None),
        "longitude": data.get("coord", {}).get("lon", None),
        "rain_1h": data.get("rain", {}).get(
            "1h", None
        ), # Précipitation sur les trois dernières heures
    }

    # Convertir le dictionnaire en DataFrame
    df_cleaned = pd.DataFrame([cleaned_data])

    return df_cleaned

In [10]:
# Parcourir la liste de villes pour récupérer les données météo et les nettoyer
for city in CITIES:
    fetch_weather_data(city)

    # Nettoyer les données pour chaque ville
    file_path = f"data/{city}.json"
    try:
        df = clean_weather_data(file_path)
        # Sauvegarder les données nettoyées dans un fichier CSV
        df.to_csv(f"data_clean/{city}.csv", index=False)
        print(f"Data cleaned and saved for {city}.")
    except Exception as e:
        print(f"Error processing data for {city}: {e}")

Weather data for Dakar saved.
Data cleaned and saved for Dakar.
Weather data for Thies saved.
Data cleaned and saved for Thies.
Weather data for Diourbel saved.
Data cleaned and saved for Diourbel.
Weather data for Fatick saved.
Data cleaned and saved for Fatick.
Weather data for Saint Louis saved.
Data cleaned and saved for Saint Louis.
Weather data for Louga saved.
Data cleaned and saved for Louga.
Weather data for Matam saved.
Data cleaned and saved for Matam.
Weather data for Kolda saved.
Data cleaned and saved for Kolda.
Weather data for Kedougou saved.
Data cleaned and saved for Kedougou.
Weather data for Sedhiou saved.
Data cleaned and saved for Sedhiou.
Weather data for Kaffrine saved.
Data cleaned and saved for Kaffrine.
Weather data for Ziguinchor saved.
Data cleaned and saved for Ziguinchor.
Weather data for Kaolack saved.
Data cleaned and saved for Kaolack.
Weather data for Tambacounda saved.
Data cleaned and saved for Tambacounda.


# Chargement dans une base de donnees

In [16]:
def load_data_to_mysql(file_path):
    # Connexion à la base de données MySQL
    conn = mysql.connector.connect(
        host="127.0.0.1",
        user="boot",
        password="",
        database="test",
        port="3306",
        charset="utf8mb4",
        collation="utf8mb4_unicode_ci",
    )
    cur = conn.cursor()

    # Nettoyer les données météo à partir du fichier JSON
    df_cleaned = clean_weather_data(file_path)

    # Insérer chaque ligne nettoyée dans la table MySQL
    for index, row in df_cleaned.iterrows():
        cur.execute(
            "INSERT INTO engine (city, temperature, humidity, pressure, weather_condition, wind_speed, wind_deg, clouds, latitude, longitude, rain_1h) "
            "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            (
                row["city"],
                row["temperature"],
                row["humidity"],
                row["pressure"],
                row["weather_condition"],
                row["wind_speed"],
                row["wind_deg"],
                row["clouds"],
                row["latitude"],
                row["longitude"],
                row["rain_1h"],
            ),
        )

    # Sauvegarder les modifications dans la base de données
    conn.commit()
    cur.close()
    conn.close()
    print(f"Data from {file_path} loaded into MySQL.")

# Traiter tous les fichiers JSON dans le dossier "data/"
def load_all_json_to_mysql(directory):
    # Parcourir tous les fichiers JSON dans le répertoire spécifié
    for filename in os.listdir(directory):
        if filename.endswith(".json"):  # Vérifier que le fichier est un JSON
            file_path = os.path.join(directory, filename)
            load_data_to_mysql(file_path)

# Appel de la fonction pour charger tous les fichiers JSON nettoyés dans MySQL
load_all_json_to_mysql("data/")

Data from data/Thies.json loaded into MySQL.
Data from data/Diourbel.json loaded into MySQL.
Data from data/Saint Louis.json loaded into MySQL.
Data from data/Kaffrine.json loaded into MySQL.
Data from data/Tambacounda.json loaded into MySQL.
Data from data/Kolda.json loaded into MySQL.
Data from data/Ziguinchor.json loaded into MySQL.
Data from data/Kedougou.json loaded into MySQL.
Data from data/Kaolack.json loaded into MySQL.
Data from data/Fatick.json loaded into MySQL.
Data from data/Dakar.json loaded into MySQL.
Data from data/Louga.json loaded into MySQL.
Data from data/Sedhiou.json loaded into MySQL.
Data from data/Matam.json loaded into MySQL.
